In [47]:
import sys
import pandas as pd
import folium
from folium.plugins import HeatMap
from utils.request import get_data

# Population
This notebook is intended to show the population of the world splitted by a country from data got from free API, and show this data in a map.

## Getting of population data

In [2]:
# Getting data from endpoint
code, data = get_data('countries')

In [3]:
# Dont continue if previous task did not retrieve data.
if code!=200:
    sys.exit(1)

In [4]:
# Transforming to pandas dataframe
df_countries = pd.DataFrame(data)

# Exploring the data in a simple way

In [5]:
pd.set_option('display.max_columns', None)
df_countries.head(2)

,name,tld,cca2,ccn3,cca3,independent,status,unMember,currencies,idd,capital,altSpellings,region,languages,translations,latlng,landlocked,area,demonyms,flag,maps,population,car,timezones,continents,flags,coatOfArms,startOfWeek,capitalInfo,cioc,subregion,fifa,borders,gini,postalCode
0,"{'common': 'South Georgia', 'official': 'South...",[.gs],GS,239,SGS,False,officially-assigned,False,"{'SHP': {'name': 'Saint Helena pound', 'symbol...","{'root': '+5', 'suffixes': ['00']}",[King Edward Point],"[GS, South Georgia and the South Sandwich Isla...",Antarctic,{'eng': 'English'},{'ara': {'official': 'جورجيا الجنوبية وجزر سان...,"[-54.5, -37.0]",False,3903.0,{'eng': {'f': 'South Georgian South Sandwich I...,🇬🇸,{'googleMaps': 'https://goo.gl/maps/mJzdaBwKBb...,30,"{'signs': [''], 'side': 'right'}",[UTC-02:00],[Antarctica],"{'png': 'https://flagcdn.com/w320/gs.png', 'sv...",{},monday,"{'latlng': [-54.28, -36.5]}",NaN,NaN,NaN,NaN,NaN,NaN
1,"{'common': 'Grenada', 'official': 'Grenada', '...",[.gd],GD,308,GRD,True,officially-assigned,True,"{'XCD': {'name': 'Eastern Caribbean dollar', '...","{'root': '+1', 'suffixes': ['473']}",[St. George's],[GD],Americas,{'eng': 'English'},"{'ara': {'official': 'غرينادا', 'common': 'غري...","[12.11666666, -61.66666666]",False,344.0,"{'eng': {'f': 'Grenadian', 'm': 'Grenadian'}, ...",🇬🇩,{'googleMaps': 'https://goo.gl/maps/rqWyfUAt4x...,112519,"{'signs': ['WG'], 'side': 'left'}",[UTC-04:00],[North America],"{'png': 'https://flagcdn.com/w320/gd.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [32.38, -64.68]}",GRN,Caribbean,GRN,NaN,NaN,NaN


In [6]:
# Confirming "regions" is equal to continent
df_countries.region.drop_duplicates()

0    Antarctic
1     Americas
2       Europe
3       Africa
5         Asia
6      Oceania
Name: region, dtype: object

In [7]:
# Getting metadata from dataframe
df_countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          250 non-null    object 
 1   tld           249 non-null    object 
 2   cca2          250 non-null    object 
 3   ccn3          249 non-null    object 
 4   cca3          250 non-null    object 
 5   independent   249 non-null    object 
 6   status        250 non-null    object 
 7   unMember      250 non-null    bool   
 8   currencies    247 non-null    object 
 9   idd           250 non-null    object 
 10  capital       246 non-null    object 
 11  altSpellings  250 non-null    object 
 12  region        250 non-null    object 
 13  languages     249 non-null    object 
 14  translations  250 non-null    object 
 15  latlng        250 non-null    object 
 16  landlocked    250 non-null    bool   
 17  area          250 non-null    float64
 18  demonyms      249 non-null    

# Applying some transformations

In [8]:
df_countries['name']=df_countries.name.map(lambda x:x['official'])
# Breaking down the field "latlng"
df_countries[['lat','lng']] = pd.DataFrame(df_countries.latlng.tolist(), index=df_countries.index)

In [9]:
columns = ['region','name','cca2','cca3','lat','lng','area','population']
df_countries = df_countries[columns]
df_countries.head(2)

,region,name,cca2,cca3,lat,lng,area,population
0,Antarctic,South Georgia and the South Sandwich Islands,GS,SGS,-54.500000,-37.000000,3903.0,30
1,Americas,Grenada,GD,GRD,12.116667,-61.666667,344.0,112519


# First 10 countries with more population

In [72]:
df_countries.sort_values(by=['population'], ascending=False)[:10]

,region,name,cca2,cca3,lat,lng,area,population
28,Asia,People's Republic of China,CN,CHN,35.0,105.0,9706961.0,1402112000
85,Asia,Republic of India,IN,IND,20.0,77.0,3287590.0,1380004385
203,Americas,United States of America,US,USA,38.0,-97.0,9372610.0,329484123
13,Asia,Republic of Indonesia,ID,IDN,-5.0,120.0,1904569.0,273523621
74,Asia,Islamic Republic of Pakistan,PK,PAK,30.0,70.0,881912.0,220892331
222,Americas,Federative Republic of Brazil,BR,BRA,-10.0,-55.0,8515767.0,212559409
214,Africa,Federal Republic of Nigeria,NG,NGA,10.0,8.0,923768.0,206139587
40,Asia,People's Republic of Bangladesh,BD,BGD,24.0,90.0,147570.0,164689383
104,Europe,Russian Federation,RU,RUS,60.0,100.0,17098242.0,144104080
24,Americas,United Mexican States,MX,MEX,23.0,-102.0,1964375.0,128932753


# Showing results in a map

In [62]:
# Initializing a folium Map 
world_map = folium.Map(
    location=[df_countries.lat.mean(), df_countries.lng.mean()], 
    zoom_start=2
)
tabla="""
    <strong>{name}</strong>
    <table>
        <tr>
            <td>Area:</td>
            <td>{area}</td>
        </tr>
        <tr>
            <td>Population:</td>
            <td>{popu} ({popum}M)</td>
        </tr>
    </table>
"""
# Define the HTML template for the icon
icon_html = '''
    <div style="background-color:red; border-radius:50%; width:12px; height:12px; opacity:0.1;"></div>
'''

# Create a custom icon
icon = folium.DivIcon(html=icon_html)

# Add markers to the map
heat_data = []
for _, row in df_countries.iterrows():
    des=tabla.format(name=row['name'], area=row['area'], 
                     popu=row['population'], 
                     popum=round(row['population']/1000000,2))
    heat_data.append([row['lat'], row['lng'], row['population']])
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=des,
        tooltip=des,
    ).add_to(world_map)
HeatMap(heat_data).add_to(world_map)
# Save the map to an HTML format
world_map.save("dataframe_map.html")

# it is not the best presentation but is good to show simple results in a graphical way!
world_map
